In [4]:
import torch
from torch import nn
from torch.nn import functional as F

X = torch.randn(2, 20)

class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))


In [5]:
net = MLP()
net(X)


tensor([[ 0.1694,  0.3024,  0.0234,  0.6028, -0.1510, -0.0793, -0.0600,  0.7704,
          0.1347, -0.0031],
        [ 0.2219,  0.1995,  0.1849, -0.0683,  0.0675,  0.1056, -0.0531,  0.3363,
         -0.1570, -0.2424]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)


tensor([[-0.1373, -0.0651, -0.2458, -0.3771, -0.0365, -0.2246,  0.1632, -0.1833,
         -0.1528, -0.6117],
        [ 0.0634,  0.0301, -0.3615, -0.0950,  0.1018,  0.0864, -0.0130, -0.2848,
          0.2015, -0.5008]], grad_fn=<AddmmBackward0>)

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)

        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


In [10]:
net = FixedHiddenMLP()
net(X)


tensor(-0.0962, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(-0.1303, grad_fn=<SumBackward0>)